# Centralized Solution of Multi-Regional Transmission Expansion Horizontal Investment Coordination Problem

This notebook contains code for a three zone test case. The case consists of an artificial merging of IEEE 14 nodes, IEEE 30 nodes, and a 5 node test system, respectively, for each zone. This notebook assumes there is a single entity (e.g. national planner) to solve the optimization problem centrally.
Results from this notebook serve a benchmark for the decentralized algorithms. 


In [3]:
# Load the different package dependencies for Julia
# Load or comment out the appropriate optimization engine (E.g. Gurobi, GLPK)

import Pkg
using Pkg
using JuMP
using GLPK
#using Gurobi
using XLSX
using DataFrames

## Problem set up

Set up the example problem by reading the input data files from the `../Input_Data/` folder, then separate the data frames for each zone.

The input data includes: 

* `CandLine.csv` which contains information on shared _candidate lines_ between zones
* `CandLineInt.csv` which contains information on internal _candidate lines_ within each zone
* `SharedEline.csv` which contains information on existing _shared lines_ between zones
* `Tran.csv` which contains information on existing transmission lines within each zone (i.e. as defined in the cited IEEE test systems)
* `Gen.csv` which contains information on the generators as defined in the cited IEEE test systems
* `Load.csv` which contains information on the loads as defined in the cited IEEE test systems

After loading the input data, construct several DataFrames for each zone, i.e. for each set of lines, generators, loads, etc.


In [7]:
# Read input files to set the problem

shared_cand = DataFrame(XLSX.readtable("../Input_Data/CandLine.csv", "Taul1", header=true)) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("../Input_Data/CandLineInt.csv", "Taul1", header=true)) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("../Input_Data/SharedEline.csv", "Taul1", header=true)) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("../Input_Data/Tran.csv", "Taul1", header=true)) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("../Input_Data/Gen.csv" , "Taul1", header=true)) #Dataframe of generators
load =  DataFrame(XLSX.readtable("../Input_Data/Load.csv", "Taul1", header=true)) #Dataframe of loads

Row,zoneNum,lNodeID,P_load1,P_load2,P_load3,P_load4
,Any,Any,Any,Any,Any,Any
1,1,2,-21.7,-10.85,-16.275,-5.425
2,1,3,-94.2,-47.1,-70.65,-23.55
3,1,4,-47.8,-23.9,-35.85,-11.95
4,1,5,-7.6,-3.8,-5.7,-1.9
5,1,6,-11.2,-5.6,-8.4,-2.8
6,1,9,-29.5,-14.75,-22.125,-7.375
7,1,10,-9,-4.5,-6.75,-2.25
8,1,11,-3.5,-1.75,-2.625,-0.875
9,1,12,-6.1,-3.05,-4.575,-1.525


In [5]:
# Create separate dataframes for each element of each zone 

l(i) = load[load.zoneNum .== i, :] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i

bin_e (generic function with 1 method)

#### Constructing the optimization model with a specified solver

In [5]:
Mod3 = Model(Gurobi.Optimizer)

Academic license - for non-commercial use only - expires 2023-09-07


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [7]:
node_number = 3;

In [8]:
@variable(Mod3,0 <= gen_var_1[1:14]) # generation at node 1
@variable(Mod3,0 <= gen_var_2[1:30]) # generation at node 2
@variable(Mod3,0 <= gen_var_3[1:5]) # generation at node 3

@variable(Mod3,shared_line_decision_var[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines

@variable(Mod3,int_line_decision_var_1[1:nrow(int_c(1))], Bin) #Decision variable for internal candidate lines of zone 1 (Binary)
@variable(Mod3,int_line_decision_var_2[1:nrow(int_c(2))], Bin) #Decision variable for internal candidate lines of zone 2 (Binary)
@variable(Mod3,int_line_decision_var_3[1:nrow(int_c(3))], Bin) #Decision variable for internal candidate lines of zone 3 (Binary)

@variable(Mod3,shared_cand_flow[1:nrow(shared_cand)])  #Power flowing on shared candidate lines 

@variable(Mod3, int_cand_flow_1[1:nrow(int_c(1))]) #Power flowing on internal candidate lines of zone 1 
@variable(Mod3, int_cand_flow_2[1:nrow(int_c(2))]) #Power flowing on internal candidate lines of zone 2
@variable(Mod3, int_cand_flow_3[1:nrow(int_c(3))]) #Power flowing on internal candidate lines of zone 3

@variable(Mod3,0 <= shared_cand_angle[1:nrow(shared_cand),1:2]<= 2*pi) #Phase angle decision for shared candidate lines

@variable(Mod3,0 <= int_cand_angle_1[1:nrow(int_c(1)),1:2]<= 2*pi) #Phase angle decision for internal candidate lines of zone 1 
@variable(Mod3,0 <= int_cand_angle_2[1:nrow(int_c(2)),1:2]<= 2*pi) #Phase angle decision for internal candidate lines of zone 2
@variable(Mod3,0 <= int_cand_angle_3[1:nrow(int_c(3)),1:2]<= 2*pi) #Phase angle decision for internal candidate lines of zone 3 

@variable(Mod3, shared_ex_flow[1:nrow(shared_ex)])  #Power flowing on shared existing shared lines

@variable(Mod3, int_ex_flow_1[1:nrow(int_e(1))])  #Power flowing on internal existing lines of zone 1 
@variable(Mod3, int_ex_flow_2[1:nrow(int_e(2))])  #Power flowing on internal existing lines of zone 2
@variable(Mod3, int_ex_flow_3[1:nrow(int_e(3))])  #Power flowing on internal existing lines of zone 3

@variable(Mod3,0 <= shared_ex_angle[1:nrow(shared_ex),1:2]<= 2*pi) #Phase angle for existing shared lines

@variable(Mod3,0 <= int_ex_angle_1[1:nrow(int_e(1)),1:2]<= 2*pi) #Phase angle decision for existing internal lines of zone 1
@variable(Mod3,0 <= int_ex_angle_2[1:nrow(int_e(2)),1:2]<= 2*pi) #Phase angle decision for existing internal lines of zone 2
@variable(Mod3,0 <= int_ex_angle_3[1:nrow(int_e(3)),1:2]<= 2*pi) #Phase angle decision for existing internal lines of zone 3


7×2 Array{VariableRef,2}:
 int_ex_angle_3[1,1]  int_ex_angle_3[1,2]
 int_ex_angle_3[2,1]  int_ex_angle_3[2,2]
 int_ex_angle_3[3,1]  int_ex_angle_3[3,2]
 int_ex_angle_3[4,1]  int_ex_angle_3[4,2]
 int_ex_angle_3[5,1]  int_ex_angle_3[5,2]
 int_ex_angle_3[6,1]  int_ex_angle_3[6,2]
 int_ex_angle_3[7,1]  int_ex_angle_3[7,2]

In [9]:
@expression(Mod3, total_cost , sum((gen_var_1[n] .* sum((g(1).gNodeID .== n) .* MC(1))) for n in 1:14)
    .+ sum((gen_var_2[n] .* sum((g(2).gNodeID .== n) .* MC(2))) for n in 1:30)
    .+ sum((gen_var_3[n] .* sum((g(3).gNodeID .== n) .* MC(3))) for n in 1:5)
    .+ sum(shared_line_decision_var[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
    .+ sum(int_line_decision_var_1[c] .* int_c(1).costPerCap[c] .* int_c(1).interestRate[c] 
            .*((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c]) ./ (((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c])-1) for c in 1:nrow(int_c(1)))
    .+ sum(int_line_decision_var_2[c] .* int_c(2).costPerCap[c] .* int_c(2).interestRate[c] 
            .*((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c]) ./ (((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c])-1) for c in 1:nrow(int_c(2)))
    .+ sum(int_line_decision_var_3[c] .* int_c(3).costPerCap[c] .* int_c(3).interestRate[c] 
            .*((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c]) ./ (((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c])-1) for c in 1:nrow(int_c(3))))   

34.30293932 gen_var_1[1] + 55 gen_var_1[2] + 3.6 gen_var_2[1] + 3.15 gen_var_2[2] + 3.5 gen_var_2[13] + 3.667 gen_var_2[22] + 3.75 gen_var_2[23] + 4.375 gen_var_2[27] + 16.30293932 gen_var_3[1] + 37 gen_var_3[2] + 0.2410176361372405 shared_line_decision_var[1] + 0.19594024895900847 shared_line_decision_var[2] + 0.44297470440637654 shared_line_decision_var[3] + 0.6532451422553538 shared_line_decision_var[4] + 0.1533748593600621 shared_line_decision_var[5] + 0.34289543567826486 shared_line_decision_var[6] + 0.1533748593600621 int_line_decision_var_1[1] + 0.34289543567826486 int_line_decision_var_1[2] + 0.2519729832343877 int_line_decision_var_1[3] + 0.21553427385490934 int_line_decision_var_1[4] + 0.44297470440637654 int_line_decision_var_1[5] + 6.532451422553539 int_line_decision_var_1[6] + 1.5337485936006214 int_line_decision_var_1[7] + 3.673879667981409 int_line_decision_var_1[8] + 2.355399625886668 int_line_decision_var_1[9] + 2.400268049747854 int_line_decision_var_1[10] + 1.5337485

In [10]:
#The first zone
for n in 1:14
        # Power balance constraint for each node
    @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n] .+ sum(l(1).P_load1 .* (l(1).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(1) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(1) .* shared_cand_flow) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(1) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(1) .* shared_ex_flow) .+
        sum((int_c(1).tNodeID1 .== n) .* int_cand_flow_1) .- sum((int_c(1).tNodeID2 .== n) .* int_cand_flow_1) .+
        sum((int_e(1).tNodeID1 .== n) .* int_ex_flow_1) .- sum((int_e(1).tNodeID2 .== n) .* int_ex_flow_1))
        #Lower limit for generation of each node
    @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n] .<= sum((g(1).gNodeID .== n) .* g(1).PgMax))
        #Upper limit for generation of each node
    @constraint(Mod3, sum((g(1).gNodeID .== n) .* g(1).PgMin) .<= sum(g(1).gNodeID .== n) .* gen_var_1[n])
end

In [11]:
#The second zone
for n in 1:30
        # Power balance constraint for each node
    @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n] .+ sum(l(2).P_load1 .* (l(2).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(2) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(2) .* shared_cand_flow) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(2) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(2) .* shared_ex_flow) .+
        sum((int_c(2).tNodeID1 .== n) .* int_cand_flow_2) .- sum((int_c(2).tNodeID2 .== n) .* int_cand_flow_2) .+
        sum((int_e(2).tNodeID1 .== n) .* int_ex_flow_2) .- sum((int_e(2).tNodeID2 .== n) .* int_ex_flow_2))
        #Lower limit for generation of each node
    @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n] .<= sum((g(2).gNodeID .== n) .* g(2).PgMax))
        #Upper limit for generation of each node
    @constraint(Mod3, sum((g(2).gNodeID .== n) .* g(2).PgMin) .<= sum(g(2).gNodeID .== n) .* gen_var_2[n])
end

In [12]:
#The third zone
for n in 1:5
        # Power balance constraint for each node
    @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n] .+ sum(l(3).P_load1 .* (l(3).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(3) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(3) .* shared_cand_flow) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(3) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(3) .* shared_ex_flow) .+
        sum((int_c(3).tNodeID1 .== n) .* int_cand_flow_3) .- sum((int_c(3).tNodeID2 .== n) .* int_cand_flow_3) .+
        sum((int_e(3).tNodeID1 .== n) .* int_ex_flow_3) .- sum((int_e(3).tNodeID2 .== n) .* int_ex_flow_3))
        #Lower limit for generation of each node
    @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n] .<= sum((g(3).gNodeID .== n) .* g(3).PgMax))
        #Upper limit for generation of each node
    @constraint(Mod3, sum((g(3).gNodeID .== n) .* g(3).PgMin) .<= sum(g(3).gNodeID .== n) .* gen_var_3[n])
end

In [13]:
#Shared candidate lines
M = 100
for c in 1:nrow(shared_cand)
    @constraint(Mod3,-M .* (1 .- shared_line_decision_var[c]) .<= shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])))
        
    @constraint(Mod3, shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (shared_cand_angle[c,1] .- shared_cand_angle[c,2])) .<= M .* (1 .- shared_line_decision_var[c]))
        #limiting the upper bound of power flow flowing within candidate shared lines
    @constraint(Mod3, shared_cand_flow[c] .<= shared_line_decision_var[c] .*shared_cand.ptMax[c])
        #Limiting the lower bound of power flowing within the candidate shared lines
    @constraint(Mod3, -shared_cand.ptMax[c] .* shared_line_decision_var[c] .<= shared_cand_flow[c]) 
end

In [14]:
#Shared existing lines
for h in 1:nrow(shared_ex)
    @constraint(Mod3, shared_ex_flow[h] .== (1 ./ shared_ex.reacT[h]) .* (shared_ex_angle[h,1] .- shared_ex_angle[h,2]))
    @constraint(Mod3, shared_ex_flow[h] .<= shared_ex. ptMax[h])
    @constraint(Mod3, -shared_ex.ptMax[h] .<= shared_ex_flow[h])
end

In [15]:
#Zone 1 internal candidate lines
for c in 1:nrow(int_c(1))
    @constraint(Mod3,-M .* (1 .- int_line_decision_var_1[c]) .<= int_cand_flow_1[c] .- ((1 ./ int_c(1).reacT[c]) .* (int_cand_angle_1[c,1] .- int_cand_angle_1[c,2])))
    @constraint(Mod3, int_cand_flow_1[c] .- ((1 ./ int_c(1).reacT[c]) .* (int_cand_angle_1[c,1] .- int_cand_angle_1[c,2])) .<= M .* (1 .- int_line_decision_var_1[c]))
    @constraint(Mod3, int_cand_flow_1[c] .<= int_line_decision_var_1[c] .* int_c(1).ptMax[c])
    @constraint(Mod3, -int_c(1).ptMax[c] .*int_line_decision_var_1[c] .<= int_cand_flow_1[c])
end

In [16]:
#Zone 2 internal candidate lines
for c in 1:nrow(int_c(2))
    @constraint(Mod3,-M .* (1 .- int_line_decision_var_2[c]) .<= int_cand_flow_2[c] .- ((1 ./ int_c(2).reacT[c]) .* (int_cand_angle_2[c,1] .- int_cand_angle_2[c,2])))
    @constraint(Mod3, int_cand_flow_2[c] .- ((1 ./ int_c(2).reacT[c]) .* (int_cand_angle_2[c,1] .- int_cand_angle_2[c,2])) .<= M .* (1 .- int_line_decision_var_2[c]))
    @constraint(Mod3, int_cand_flow_2[c] .<= int_line_decision_var_2[c] .* int_c(2).ptMax[c])
    @constraint(Mod3, -int_c(2).ptMax[c] .*int_line_decision_var_2[c] .<= int_cand_flow_2[c])
end

In [17]:
#Zone 3 internal candidate lines
for c in 1:nrow(int_c(3))
    @constraint(Mod3,-M .* (1 .- int_line_decision_var_3[c]) .<= int_cand_flow_3[c] .- ((1 ./ int_c(3).reacT[c]) .* (int_cand_angle_3[c,1] .- int_cand_angle_3[c,2])))
    @constraint(Mod3, int_cand_flow_3[c] .- ((1 ./ int_c(3).reacT[c]) .* (int_cand_angle_3[c,1] .- int_cand_angle_3[c,2])) .<= M .* (1 .- int_line_decision_var_3[c]))
    @constraint(Mod3, int_cand_flow_3[c] .<= int_line_decision_var_3[c] .* int_c(3).ptMax[c])
    @constraint(Mod3, -int_c(3).ptMax[c] .*int_line_decision_var_3[c] .<= int_cand_flow_3[c])
end

In [18]:
#Zone 1 internal existing lines
for h in 1:nrow(int_e(1))
    @constraint(Mod3, int_ex_flow_1[h] .== (1 ./ int_e(1).reacT[h]) .* (int_ex_angle_1[h,1] .- int_ex_angle_1[h,2]))
    @constraint(Mod3, int_ex_flow_1[h] .<= int_e(1).ptMax[h])
    @constraint(Mod3, -int_e(1).ptMax[h] .<= int_ex_flow_1[h])
end

In [19]:
#Zone 2 internal existing lines
for h in 1:nrow(int_e(2))
    @constraint(Mod3, int_ex_flow_2[h] .== (1 ./ int_e(2).reacT[h]) .* (int_ex_angle_2[h,1] .- int_ex_angle_2[h,2]))
    @constraint(Mod3, int_ex_flow_2[h] .<= int_e(2).ptMax[h])
    @constraint(Mod3, -int_e(2).ptMax[h] .<= int_ex_flow_2[h])
    #@constraint(Mod3, (int_e(2).tNodeID1 .== 8) .* int_ex_angle_2[h,1] .== 0)
    #@constraint(Mod3, (int_e(2).tNodeID2 .== 8) .* int_ex_angle_2[h,2] .== 0)
end

In [20]:
#Zone 3 internal existing lines
for h in 1:nrow(int_e(3))
    @constraint(Mod3, int_ex_flow_3[h] .== (1 ./ int_e(3).reacT[h]) .* (int_ex_angle_3[h,1] .- int_ex_angle_3[h,2]))
    @constraint(Mod3, int_ex_flow_3[h] .<= int_e(3).ptMax[h])
    @constraint(Mod3, -int_e(3).ptMax[h] .<= int_ex_flow_3[h])
end

In [21]:
@objective(Mod3, Min, total_cost)
optimize!(Mod3)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 519 rows, 421 columns and 1082 nonzeros
Model fingerprint: 0x6efddade
Variable types: 385 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 6e+01]
  Bounds range     [6e+00, 6e+00]
  RHS range        [2e+00, 3e+02]
Presolve removed 341 rows and 246 columns
Presolve time: 0.00s
Presolved: 178 rows, 175 columns, 568 nonzeros
Variable types: 140 continuous, 35 integer (35 binary)

Root relaxation: objective 5.311879e+03, 115 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5311.87898    0   12          - 5311.87898      -     -    0s
H    0     0                    8675.8947927 5311.87898  38.8%     -    0s
H    0     0            

In [22]:
if termination_status(Mod3) == MOI.OPTIMAL
    shared_line_decision = value.(shared_line_decision_var)
    shared_cand_power = value.(shared_cand_flow)
    shared_cand_phase_angle = value.(shared_cand_angle)
    shared_ex_power = value.(shared_ex_flow)
    shared_ex_phase_angle = value.(shared_ex_angle)
    int_line_decision_1 = value.(int_line_decision_var_1)
    int_line_decision_2 = value.(int_line_decision_var_2)
    int_line_decision_3 = value.(int_line_decision_var_3)
    int_line_flow_1 = value.(int_cand_flow_1)
    int_line_flow_2 = value.(int_cand_flow_2)
    int_line_flow_3 = value.(int_cand_flow_3)
    generation_1 = value.(gen_var_1)
    generation_2 = value.(gen_var_2)
    generation_3 = value.(gen_var_3)
    obj_value = objective_value(Mod3)
end

5318.843619397567

In [23]:
shared_line_decision

6-element Array{Float64,1}:
  1.0
 -0.0
  1.0
  1.0
  1.0
  1.0

In [24]:
shared_cand_power

6-element Array{Float64,1}:
  57.72798824241809
   0.0
  31.57378014035332
 100.00000000000004
  38.03380936549385
 -34.37502458623578

In [25]:
shared_cand_phase_angle

6×2 Array{Float64,2}:
 3.41577   0.0
 6.28319   0.0
 6.25066   0.0
 5.75      0.0
 6.28319   0.0
 0.312244  6.28319

In [26]:
shared_ex_phase_angle

8×2 Array{Float64,2}:
 6.28319  0.0
 0.0      6.28319
 0.0      0.0
 6.28319  0.0
 2.18084  0.0
 6.28319  0.0
 6.28319  3.81037
 1.41576  0.0

In [27]:
shared_ex_power

8-element Array{Float64,1}:
  23.271056693257723
 -19.086225112939207
   0.0
  11.300692998524434
  36.407990631580645
  31.41592653589793
  22.48013940828421
   6.806531259561535

In [28]:
int_line_decision_1

10-element Array{Float64,1}:
 -0.0
  1.0
  1.0
 -0.0
 -0.0
  1.0
 -0.0
 -0.0
 -0.0
  1.0

In [29]:
int_line_decision_2

18-element Array{Float64,1}:
  1.0
 -0.0
 -0.0
 -0.0
 -0.0
  1.0
  1.0
 -0.0
 -0.0
  1.0
  1.0
 -0.0
 -0.0
 -0.0
  1.0
 -0.0
 -0.0
 -0.0

In [30]:
int_line_decision_3

2-element Array{Float64,1}:
 -0.0
 -0.0

In [31]:
obj_value

5318.843619397568

In [31]:
generation_1

14-element Vector{Float64}:
 135.72894330674225
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0

In [32]:
show(stdout, "text/plain", generation_2)

30-element Vector{Float64}:
 69.20000000000002
 79.99999999999999
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 40.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [33]:
generation_3

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [32]:
int_line_flow_1

10-element Vector{Float64}:
   0.0
  35.635125381009445
 -36.13518120071081
   0.0
   0.0
 -30.045836396229845
   0.0
   0.0
   0.0
  13.761720136141406

In [36]:
int_line_flow_2

18-element Vector{Float64}:
 -83.35901940310741
   0.0
   0.0
   0.0
   0.0
 -31.621466065322522
 -31.46312121772452
   0.0
   0.0
 -32.67387055215593
 -47.73050620457735
   0.0
   0.0
   0.0
  19.086225112939204
   0.0
   0.0
   0.0

In [37]:
int_line_flow_3

2-element Vector{Float64}:
 0.0
 0.0